In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

In [29]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor 

### Reading csv file using Pandas

In [89]:
df=pd.read_csv("C:/Users/naren/OneDrive/Desktop/EndToEnd/notebook/data/StudentsPerformance.csv")
df

gender race/ethnicity parental level of education         lunch  \
0    female        group B           bachelor's degree      standard   
1    female        group C                some college      standard   
2    female        group B             master's degree      standard   
3      male        group A          associate's degree  free/reduced   
4      male        group C                some college      standard   
..      ...            ...                         ...           ...   
995  female        group E             master's degree      standard   
996    male        group C                 high school  free/reduced   
997  female        group C                 high school  free/reduced   
998  female        group D                some college      standard   
999  female        group D                some college  free/reduced   

    test preparation course  math score  reading score  writing score  
0                      none          72             72             74  
1                 completed          69             90             88  
2                      none          90             95             93  
3                      none          47             57             44  
4                      none          76             78             75  
..                      ...         ...            ...            ...  
995               completed          88             99             95  
996                    none          62             55             55  
997               completed          59             71             65  
998               completed          68             78             77  
999                    none          77             86             86  

[1000 rows x 8 columns]

In [90]:
X=df.drop(columns=["reading score","writing score"])
y=(df["math score"]+df["reading score"]+df["writing score"])
columns=X.columns
X.head()

gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  
0                    none          72  
1               completed          69  
2                    none          90  
3                    none          47  
4                    none          76

# Encoding categorical data using Column Transformer 

In [91]:
numerical=[col for col in X.columns if (df[col].dtype == 'int64')]
print(f"numerical data: {numerical}")
categorical=[col for col in X.columns if col not in numerical]
print(f"categorical data: {categorical} ")

numerical data: ['math score']
categorical data: ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course'] 


In [92]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
encoder = OneHotEncoder()
from sklearn.compose import ColumnTransformer
transform =ColumnTransformer(
   [ ("encoding:" , encoder, categorical ),
     ("scaling:" , scaler , numerical)
     ]
)

X=transform.fit_transform(X)

In [93]:
X.shape

(1000, 18)

# Train Test split

In [94]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)
X_train.shape

(800, 18)

In [95]:
def evaluate(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [96]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 10.3121
- Mean Absolute Error: 8.2411
- R2 Score: 0.9408
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10.1782
- Mean Absolute Error: 8.0575
- R2 Score: 0.9463


Lasso
Model performance for Training set
- Root Mean Squared Error: 11.3706
- Mean Absolute Error: 9.1938
- R2 Score: 0.9280
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 11.2345
- Mean Absolute Error: 8.7360
- R2 Score: 0.9346


Ridge
Model performance for Training set
- Root Mean Squared Error: 10.3124
- Mean Absolute Error: 8.2415
- R2 Score: 0.9408
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10.1817
- Mean Absolute Error: 8.0622
- R2 Score: 0.9463


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 15.6248
- Mean Absolute Error: 12.5515
- R2 Score: 0.8640
---------------

In [97]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


Model Name  R2_Score
0        Linear Regression  0.946304
2                    Ridge  0.946267
1                    Lasso  0.934581
7    CatBoosting Regressor  0.928084
5  Random Forest Regressor  0.923056
6             XGBRegressor  0.916848
8       AdaBoost Regressor  0.912690
4            Decision Tree  0.888823
3    K-Neighbors Regressor  0.815686

In [98]:
model=LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [ ]:
# Example input row (match column order exactly)
X_new = pd.DataFrame([{
    "gender": "female",
    "race/ethnicity": "group A",
    "parental level of education": "master's degree",
    "lunch": "free/reduced",
    "test preparation course": "completed",
    "math score": 
}])

# Apply the same encoder/transformer as during training
X_new_encoded = transform.transform(X_new)

# Now predict
percentage=(model.predict(X_new_encoded)/300)*100
print("Percentage-->", percentage)            


Percentage--> [79.15169733]
